# 基于位置方差的机器人运动学参数标定

目标函数:最小化观测点位置方差
目标精度:位置标准差1mm以内

In [1]:
import torch
import numpy as np


print("=== Position Variance-Based Robot Kinematics Parameter Calibration Demo ===")
print("Objective Function: Minimize variance of observed positions")
print("Target precision: Position standard deviation < 1 mm")

=== Position Variance-Based Robot Kinematics Parameter Calibration Demo ===
Objective Function: Minimize variance of observed positions
Target precision: Position standard deviation < 1 mm


## 1. 创建机器人模型

In [2]:
from data_process.robot_model import create_robot
import numpy as np

# 使用现成的机器人创建函数
robot_name = "FR16"  # 设置机器人名称
robot = create_robot(robot_name=robot_name)

print(f"Created robot: {robot.name}")
print(f"Robot DOF: {robot.n}")

# 打印机器人参数
print("\nRobot DH Parameters:")
for i, link in enumerate(robot.links):
    print(f"Link {i+1}: a={link.a.item():.1f}mm, d={link.d.item():.1f}mm, "
          f"alpha={np.degrees(link.alpha.item()):.1f}°, beta={np.degrees(link.beta.item()):.1f}°, "
          f"theta_offset={np.degrees(link.offset.item()):.1f}°")

# 打印base变换
print("\nBase Transform (World to Robot Base):")
if hasattr(robot, 'base') and robot.base is not None:
    base_pos = robot.base[0:3, 3]
    print(f"  Translation: x={base_pos[0]:.1f}mm, y={base_pos[1]:.1f}mm, z={base_pos[2]:.1f}mm")
    print(f"  Base transform shape: {robot.base.shape}")
else:
    print("  No base transform defined")

# 打印手眼参数(现在是可校准的)
print("\nHand-Eye Parameters (Calibratable):")
print(f"  Translation: tx={robot.tx.item():.1f}mm, ty={robot.ty.item():.1f}mm, tz={robot.tz.item():.1f}mm")
print(f"  Rotation: rx={robot.rx.item():.3f}rad, ry={robot.ry.item():.3f}rad, rz={robot.rz.item():.3f}rad")
print(f"  Tool transform shape: {robot.tool.shape}")

# 测试前向运动学
q_test = [0.1, 0.2, 0.3, 0.1, 0.2, 0.1]  # 6个关节角度
print(f"\nTest joint angles: {q_test}")
T_test = robot.fkine(q_test)
print(f"Forward kinematics test passed")
print(f"End effector position: {T_test[0:3, 3].detach().numpy()}")
print(f"End effector position (mm): {T_test[0:3, 3].detach().numpy()}")

Created robot: FR16_Calibration_Robot
Robot DOF: 6

Robot DH Parameters:
Link 1: a=-0.0mm, d=180.0mm, alpha=90.0°, beta=0.0°, theta_offset=-0.0°
Link 2: a=520.0mm, d=0.0mm, alpha=0.0°, beta=0.0°, theta_offset=180.0°
Link 3: a=400.0mm, d=0.0mm, alpha=0.0°, beta=0.0°, theta_offset=0.0°
Link 4: a=0.0mm, d=159.0mm, alpha=-90.0°, beta=0.0°, theta_offset=-0.0°
Link 5: a=0.0mm, d=114.0mm, alpha=90.0°, beta=0.0°, theta_offset=0.0°
Link 6: a=0.0mm, d=106.0mm, alpha=-0.0°, beta=0.0°, theta_offset=180.0°

Base Transform (World to Robot Base):
  Translation: x=0.0mm, y=0.0mm, z=0.0mm
  Base transform shape: torch.Size([4, 4])

Hand-Eye Parameters (Calibratable):
  Translation: tx=0.0mm, ty=0.0mm, tz=0.0mm
  Rotation: rx=0.000rad, ry=0.000rad, rz=0.000rad
  Tool transform shape: torch.Size([4, 4])

Test joint angles: [0.1, 0.2, 0.3, 0.1, 0.2, 0.1]
Forward kinematics test passed
End effector position: [-783.3691845  -342.80608196 -221.05730464]
End effector position (mm): [-783.3691845  -342.8060819

## 2.手眼预标定

In [3]:
from eye_hand_pre_cali.pre_calibrator import PreCalibrator
from data_process.preprocess import load_dataset, create_observations
import os

print("=== 手眼预标定 ===")

# 创建预标定器实例
pre_calibrator = PreCalibrator()

# 配置数据路径和参数
data_dir = os.path.join(os.getcwd(), "data", "hand_eye_data")
num_samples = 12
poses_file = "poses.json"
clouds_subdir = "clouds"

print("正在加载和预处理数据...")
print(f"数据路径: {data_dir}")
print(f"样本数量: {num_samples}")
print(f"位姿文件: {poses_file}")
print(f"点云子目录: {clouds_subdir}")

# 检查数据路径是否存在
if not os.path.exists(data_dir):
    print(f"❌ 数据文件夹 {data_dir} 不存在")
else:
    try:
        # 加载数据集
        print("正在加载数据集...")
        poses, point_clouds = load_dataset(data_dir, num_samples=num_samples, 
                                         poses_file=poses_file, clouds_subdir=clouds_subdir)

        if poses is None or point_clouds is None:
            print("❌ 数据集加载失败")
        else:
            # 创建观测数据
            print("正在创建观测数据...")
            valid_poses = [p for p in poses if p is not None]
            valid_clouds = [pc for pc in point_clouds if pc is not None]
            observations_he = create_observations(valid_poses, valid_clouds)

            if len(observations_he) < num_samples:
                print(f"❌ 观测数据不足，需要至少{num_samples}组，当前只有{len(observations_he)}组")
            else:
                print(f"✓ 成功创建 {len(observations_he)} 个观测数据")

                # 设置观测数据到标定器
                print("正在设置观测数据到预标定器...")
                if pre_calibrator.set_observations(observations_he):
                    # 执行预标定
                    print("开始执行手眼预标定...")
                    pre_result = pre_calibrator.pre_calibrator()

                    if pre_result:
                        print("\n=== 手眼预标定结果 ===")
                        print(f"平移: x={pre_result['translation'][0]:.3f}, y={pre_result['translation'][1]:.3f}, z={pre_result['translation'][2]:.3f} mm")
                        print(f"旋转: r={pre_result['euler_angles'][0]:.3f}, p={pre_result['euler_angles'][1]:.3f}, y={pre_result['euler_angles'][2]:.3f} rad")
                        print(f"旋转矩阵:\n{pre_result['rotation_matrix']}")
                        print(f"变换矩阵:\n{pre_result['transform_matrix']}")

                        # 调试平移计算
                        print("\n--- 调试信息 ---")
                        print(f"平移向量 (原始): {pre_result['translation']}")
                        translation_magnitude = (pre_result['translation'][0]**2 + pre_result['translation'][1]**2 + pre_result['translation'][2]**2)**0.5
                        print(f"平移向量的模长: {translation_magnitude:.3f} mm")
                        print(f"平移向量的方向: x={pre_result['translation'][0]/translation_magnitude:.3f}, y={pre_result['translation'][1]/translation_magnitude:.3f}, z={pre_result['translation'][2]/translation_magnitude:.3f}")
                        print("--- 调试信息结束 ---")
                    else:
                        print("❌ 手眼预标定失败")
                else:
                    print("❌ 观测数据设置失败")

    except Exception as e:
        print(f"❌ 预标定过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
=== 手眼预标定 ===
正在加载和预处理数据...
数据路径: d:\桌面\实习\diff_cali\data\hand_eye_data
样本数量: 12
位姿文件: poses.json
点云子目录: clouds
正在加载数据集...
从 d:\桌面\实习\diff_cali\data\hand_eye_data 加载数据集...
✓ 成功读取位姿文件: poses.json, 12 个位姿
⚠️  过滤了 3015692 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00001.ply, 130036 个点
⚠️  过滤了 3015692 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00001.ply, 130036 个点
⚠️  过滤了 3003989 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00002.ply, 141739 个点
⚠️  过滤了 3003989 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00002.ply, 141739 个点
⚠️  过滤了 3002031 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00003.ply, 143697 个点
⚠️  过滤了 3002031 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00003.ply, 143697 个点
⚠️  过滤了 2983482 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00004.ply, 162246 个点
⚠️  过滤了 2983482 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00004.ply, 162246 个点
⚠️  过滤了 3001616 个包含NaN/Inf的点

## 3. 读取PLY点云文件并计算球心位置

使用项目中的 `read_ply_file` 读取点云文件，`fit_sphere_center_TLS_A` 计算球心位置，然后通过手眼关系变换到工具末端坐标系。

In [4]:
from data_process.preprocess import read_ply_file
from data_process.cloud import fit_sphere_center_TLS_A
import torch

def transform_point_to_tool(point, hand_eye_transform):
    """将相机坐标系中的点通过手眼关系变换到工具末端坐标系"""
    # 将点转换为齐次坐标
    point_homo = np.append(point, 1.0)
    
    # 使用手眼变换矩阵进行变换
    point_tool = hand_eye_transform @ point_homo
    
    # 只保留xyz坐标
    return point_tool[:3]

# 设置文件路径
ply_file_path = os.path.join(os.getcwd(), "data", "test_data_pose1", "clouds", "point_cloud_00001.ply")

print(f"=== 读取点云文件 ===")
print(f"点云文件路径: {ply_file_path}")

# 检查文件是否存在
if not os.path.exists(ply_file_path):
    print(f"❌ 点云文件不存在: {ply_file_path}")
else:
    # 使用项目中的函数读取点云数据
    print("正在使用 read_ply_file 读取点云数据...")
    point_cloud = read_ply_file(ply_file_path)
    
    if point_cloud is not None:
        print(f"✓ 成功读取点云，包含 {len(point_cloud)} 个点")
        
        # 使用 fit_sphere_center_TLS_A 计算球心位置（点云中心）
        print("正在使用 fit_sphere_center_TLS_A 计算球心位置...")
        point_cloud_tensor = torch.tensor(point_cloud, dtype=torch.float64)
        
        # 计算球心位置
        sphere_center = fit_sphere_center_TLS_A(point_cloud_tensor)
        sphere_center_np = sphere_center.detach().numpy()
        
        print(f"点云球心位置 (相机坐标系): [{sphere_center_np[0]:.3f}, {sphere_center_np[1]:.3f}, {sphere_center_np[2]:.3f}] mm")
        
        # 检查是否有手眼预标定结果
        if 'pre_result' in locals():
            print("\n=== 使用手眼预标定结果进行坐标变换 ===")
            
            # 获取手眼变换矩阵
            hand_eye_transform = pre_result['transform_matrix']
            print("正在使用手眼变换矩阵...")
            print(f"手眼变换矩阵:\n{hand_eye_transform}")
            
            # 将球心位置变换到工具末端坐标系
            print("正在变换球心位置到工具末端坐标系...")
            sphere_center_tool = transform_point_to_tool(sphere_center_np, hand_eye_transform)
            
            
            print(f"球心位置 (工具末端坐标系): [{sphere_center_tool[0]:.3f}, {sphere_center_tool[1]:.3f}, {sphere_center_tool[2]:.3f}] mm")
            
        else:
            print("❌ 未找到手眼预标定结果")
            print("请确保已运行第6个单元格进行手眼预标定")
        
    else:
        print("❌ 点云数据读取失败")

=== 读取点云文件 ===
点云文件路径: d:\桌面\实习\diff_cali\data\test_data_pose1\clouds\point_cloud_00001.ply
正在使用 read_ply_file 读取点云数据...
⚠️  过滤了 2976977 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00001.ply, 168751 个点
✓ 成功读取点云，包含 168751 个点
正在使用 fit_sphere_center_TLS_A 计算球心位置...
点云球心位置 (相机坐标系): [-35.233, 22.028, 304.539] mm

=== 使用手眼预标定结果进行坐标变换 ===
正在使用手眼变换矩阵...
手眼变换矩阵:
[[ 7.19441162e-01  6.69172037e-01  1.86046229e-01 -1.27153028e+02]
 [-6.52035997e-01  7.42999440e-01 -1.50999638e-01 -5.16470884e+01]
 [-2.39276980e-01 -1.26734834e-02  9.70868637e-01  9.65501655e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
正在变换球心位置到工具末端坐标系...
球心位置 (工具末端坐标系): [-81.103, -58.292, 400.369] mm
⚠️  过滤了 2976977 个包含NaN/Inf的点
✓ 成功读取PLY文件: point_cloud_00001.ply, 168751 个点
✓ 成功读取点云，包含 168751 个点
正在使用 fit_sphere_center_TLS_A 计算球心位置...
点云球心位置 (相机坐标系): [-35.233, 22.028, 304.539] mm

=== 使用手眼预标定结果进行坐标变换 ===
正在使用手眼变换矩阵...
手眼变换矩阵:
[[ 7.19441162e-01  6.69172037e-01  1.86046229e-01 -1.27153028e+02]
 [-6.52035997e-01 